# Constructing the Method of Moment Matrix
# by Allowing Automatic Differentiation (ForwardDiff, Zygote or Otherwise)
# to Operate Twice on Quadrature  (QuadGK, HCubature or Otherwise) Output 

## Import Packages

In [49]:
using ForwardDiff
using QuadGK
using HCubature 
using StaticArrays 
using LinearAlgebra
using BenchmarkTools
using Plots 

## Section 1: Introduction 
<b>Motivation</b> We wish to generate the MoM matrix as the linear transformation matrix (sensitivity matrix, input-output matrix) mapping magnetization vector $\vec{M}(\vec{x})$ to the magnetic flux vector $\vec{B}(\vec{x})$ after spatial discretization (first discretize, then differentiate) (need to include magnetic field $\vec{H}(\vec{x})$). In the following, we make this idea more precise. 

Assume a 3D computational domain $\vec{x} \in \Omega$ with a subset $\Omega_M$ on with the magnetic sources are defined. In a continuous setting (i.e. prior to spatial discretization), the magnetic vector potential $\vec{A}(\vec{x})$ is a function of the magnetization vector $\vec{M}(\vec{x})$. A [Biot-Savart](https://en.wikipedia.org/wiki/Biot–Savart_law) type relation yields that 
$$\vec{A}(\vec{x}) = \frac{\mu_0}{4 \pi} \int_{\Omega_m} 
                \frac{\vec{M}(\vec{x}') \times (\vec{x}-\vec{x}')}{|\vec{x}-\vec{x}'|^3} \, d\vec{x}' \, = \int_{\Omega_m} \vec{M}(\vec{x}') \times K(\vec{x}, \vec{x}') \, d\vec{x}' $$, where the kernel $K(\vec{x}, \vec{x}')$ is given by 
$$K(\vec{x},\vec{x}') = \frac{\mu_0}{4 \pi}  
                \frac{(\vec{x}-\vec{x}')}{\|\vec{x}-\vec{x}'\|^3} = 
                \frac{\mu_0}{4 \pi} \nabla \frac{1}{\|\vec{x}-\vec{x}'\|} \, . $$              The magnetic flux $\vec{B}(\vec{x})$ is then obtained via 
$$ \vec{B}(\vec{x}) = \nabla \times \vec{A}(\vec{x}) \, . $$
The magnetic field $\vec{H}(\vec{x})$ can be obtain as $\mu \, \vec{B}(\vec{x})$ in the ferromagnetic region and from $\mu \, \vec{B}(\vec{x}) + \vec{M}(\vec{x})$ in the magnetic region.  

A mesh $\Omega^h$ on $\Omega$ with $N$ nodes is defined. The numerical approximation to vector $\vec{M}(\vec{x})$ is expressed as a linear combination on shape function (or basis functions) as (three components of $\vec{M}^h(\vec{x}$ expressed in the same linear nodal first order Lagrange basis functions, the summation upper therefore denoted by $3N$)
$$ \vec{M}(\vec{x}) \thickapprox \vec{M}^h(\vec{x})= \sum_{i=1}^{3N} m_i \, \phi_i(\vec{x}) \, .$$ 
Replacing $\vec{M}(\vec{x})$ by $\vec{M}^h(\vec{x})$ in the expression for $\vec{A}(\vec{x})$ yiels the numerical approximation $\vec{A}^h(\vec{x})$. Similarly, replacing $\vec{A}(\vec{x})$ by $\vec{A}^h(\vec{x})$ in the expression for $\vec{B}(\vec{x})$, yiels the numerical approximation $\vec{B}^h(\vec{x})$. Let $\vec{m}$ denote the $3N$-vector of expansion coefficients of $\vec{M}^h(\vec{x})$ in the basis of $\phi_i{(\vec{x})}$ for $1 \leq i \leq N$ shape functions. Let $\vec{a}$ and $\vec{b}$ denote the $3N$-vectors obtained by point-matching the relationships that define them. Then clearly both $\vec{a}$ and $\vec{b}$ are vector-valued functions with $3N$ components that map $\vec{m}$ to $\vec{a}(\vec{m})$ and $\vec{b}(\vec{m})$, respectively. We can thus write formally that $\vec{a}: \vec{m} \in R^{3N} \rightarrow \vec{a}(\vec{m}) \in R^{3N}$ and  
$\vec{b}: \vec{m} \in R^{3N} \rightarrow \vec{b}(\vec{m}) \in R^{3N}$. The Jacobian of mapping 
$\partial b_i / \partial m_j$ is the MoM matrix. We wish to develop a procedure that compute this Jacobian by going through the following three steps: 
1. apply (adaptive) quadrature (including handling of singular integrals) to compute the mapping $\vec{a}(\vec{m})$, i.e., a vector function of $3N$ components $a_i$ that each depend on the $3N$ vector $\vec{a}$. The quadrature on $\Omega$ is carried out element-wise;
2. compute $\vec{b} = \nabla \times \vec{a}(\vec{x})$  
3. apply automatic differentiation to differentiate (compute the Jacobian) the magnetic flux $\vec{b}(\vec{m}) = \vec{b}(\vec{a}(\vec{m})))$ wrt to $\vec{m}$ to obtain the MoM matrix;

The above ideas are borrowed from non-linear FEM analysis is which the Jacobian is formed as the derivative of the residual vector w.r.t. the state vector. More details of both steps will be outlined below. These steps focuss on the <b>generation</b> MoM matrix. Solving of the linear system with the MoM will be discussed elsewhere.  

### Section 1.2: First Proof of Concept
Below we give a proof of concept of composing quadrature and automatic differentiation on a synthetic test case. This examples assummes a one-dimensional domain and uses smooth (harmless) integration kernel. More effort is required to treat singular kernels in three-dimensional computational domains (as required in MoM).

In [50]:
# define the magnetization
# define the kernel of integration. Here p stands p for prime. 
# define the vector potential density as magnetization times kernel 
magnetization(x,m) = m[1]*sin(π*x[1])+m[2]*cos(π*x[1]) 
kernel(x,xp) = (x[1]-xp[1])
vpdens(x,xp,m) = magnetization(xp,m)*kernel(x,xp)

# define the vector potential (denoted by vp) by integrating in the integration variable xp 
# over the magnetization domain, here assumed to be domain from 0 to 1. 
vp(x,m) = quadgk(xp -> vpdens(x,xp,m), 0, 1)
#vp(x,m) = hcubature(xp -> vpdens(x,xp,m), (0,), (1,))

# define the magnetic flux as the partial derivative of the potential wrt x (keeping m fixed)
bflux(x,m) = ForwardDiff.derivative(x -> vp(x,m)[1],x)

## define the sampling points in x (space) and m (coefficients of the expansion on magnetization in the function basis) 
xx = [.25, .75]
mm = [1., 2.]

# collocate the field on the grid in x - do point matching 
# observe depency in m remains 
bfluxsamp(m) = [bflux(xi,m) for xi in xx]

# compute the Jacobian
# to do: preallocate memory of the Jacobian 
A = ForwardDiff.jacobian(bfluxsamp, mm)

2×2 Matrix{Float64}:
 0.63662  3.77101e-17
 0.63662  3.77101e-17

### Section 2.1: Second Proof of Concept
One-dimensional test case assuming magnetization to be <b>piece-wise constant</b> or constant over each element in the mesh. Both collocation (point-matching) and Galerkin (averaged formulation) are implemented.  

To do: 
1. understand the singularity of the integrand (should be considered in 3 dimensions); 

In [66]:
# define the mesh with N elements e_k = [x_k, x_{k+1}] for 1 <= k <= N 
# the mesh contains N+1 nodes of which N-1 internal and 2 (left and right) boundary nodes  
N = 10; h = 1/N;  
xmesh = Vector(0:h:1)

# define the magnetization coefficients  
m = ones(N)  

# define integration kernel with two inputs  
epsilon = 0.01 
kernel(x,xp) = (x-xp)/(abs(x - xp)^3+epsilon) 

# compute integral by quadrature over second input (xp) over all elements e_k 
# on e_k the integral is x-dependent and stored in a array of functions 
vparray = Array{Function}(undef, N)
for k=1:N
   vparray[k] = (x,m,xmesh) -> quadgk(xp -> m[k]*kernel(x,xp), xmesh[k], xmesh[k+1])[1]
end 

# compute bflux as x-derivative over all elements of vparray 
# observe syntax used to define an array of functions
bfluxarray = Array{Function}(undef, N) 
for k=1:N
  bfluxarray[k] = (x,m,xmesh) -> ForwardDiff.derivative(x -> vparray[k](x,m,xmesh)[1],x)
end 

# sum over source location or xp contributions over N elements in the mesh  
vp(x,m,xmesh) = sum([vparray[k](x,m,xmesh) for k=1:N])
bflux(x,m,xmesh) = sum([bfluxarray[k](x,m,xmesh) for k=1:N])

# collocate the field on the grid in x - do point matching 
# observe depency in m remains 
bfluxpointsampled(m) = [bflux(xi,m,xmesh) for xi in xmesh]

#average over destination or x contributions 
bfluxaverage(m) = [quadgk(x -> bflux(x,m,xmesh), xmesh[k], xmesh[k+1])[1] for k=1:N]

# compute the Jacobian
# to do: preallocate memory of the Jacobian 
ForwardDiff.jacobian(bfluxpointsampled, m)
ForwardDiff.jacobian(bfluxaverage, m)

10×10 Matrix{Float64}:
  0.962331    0.602432   -0.112547   …  -0.0551764  -0.037732   -0.0268173
  0.602432    0.962331    0.602432      -0.0843194  -0.0551764  -0.037732
 -0.112547    0.602432    0.962331      -0.134618   -0.0843194  -0.0551764
 -0.304468   -0.112547    0.602432      -0.218104   -0.134618   -0.0843194
 -0.218104   -0.304468   -0.112547      -0.304468   -0.218104   -0.134618
 -0.134618   -0.218104   -0.304468   …  -0.112547   -0.304468   -0.218104
 -0.0843194  -0.134618   -0.218104       0.602432   -0.112547   -0.304468
 -0.0551764  -0.0843194  -0.134618       0.962331    0.602432   -0.112547
 -0.037732   -0.0551764  -0.0843194      0.602432    0.962331    0.602432
 -0.0268173  -0.037732   -0.0551764     -0.112547    0.602432    0.962331

### Section 3.1: Third Proof of Concept
Same as second example, this time using linear basis functions. 

In [68]:
# generate set of basis functions centered on node i including the boundary nodes 
# here we cheat and code an explicit representation of the linear basis functions 
# this renders the coding that follows easier 
function basisfct(x, xmesh, i)
    Np1 = length(xmesh)
    left = 0.
    right = 0.  
    if (i==1)
        right_domain = (x>=xmesh[i])*(x<=xmesh[i+1])
        right_value = (x-xmesh[i+1])/(xmesh[i]-xmesh[i+1])
        right = right_value*right_domain        
    end 
    if ((i>1)&&(i<Np1))
        left_domain = (x>=xmesh[i-1])*(x<=xmesh[i])
        left_value = (xmesh[i-1]-x)/(xmesh[i-1]-xmesh[i])
        left = left_value*left_domain        
        right_domain = (x>xmesh[i])*(x<=xmesh[i+1])
        right_value = (x-xmesh[i+1])/(xmesh[i]-xmesh[i+1])
        right = right_value*right_domain        
    end 
    if (i==Np1)
        left_domain = (x>=xmesh[i-1])*(x<=xmesh[i])
        left_value = (xmesh[i-1]-x)/(xmesh[i-1]-xmesh[i])
        left = left_value*left_domain
    end 
    result = left+right
    return result 
end 

# expand the magnetization in the bais 
function magnetization(x,m,xmesh)
    Np1 = length(xmesh)
    result = 0. 
    for i=1:Np1 
        result += m[i]*basisfct(x,xmesh,i)
    end 
    return result  
end 

magnetization (generic function with 2 methods)

In [70]:
# generate the mesh  with N elements (intervals) and N+1 nodes
N = 5; h = 1/N; Np1 = N+1; 
xmesh = Vector(0:h:1)

# set the magnetization coefficients  
m = ones(Np1) 

# define integration kernel with two inputs  
epsilon = 0.01 
kernel(x,xp) = (x-xp)/(abs(x - xp)^3+epsilon) 

# compute integral by quadrature over second input (xp) over all nodes x_k  
# on e_k the integral is x-dependent and stored in a array of functions 
vparray = Array{Function}(undef, N)
for k=1:N 
   vparray[k] = (x,m,xmesh) -> quadgk(xp -> magnetization(xp,m,xmesh)*kernel(x,xp), xmesh[k], xmesh[k+1])[1]
end 

# compute bflux as x-derivative over all elements of vparray 
bfluxarray = Array{Function}(undef, N) 
for k=1:N
  bfluxarray[k] = (x,m,xmesh) -> ForwardDiff.derivative(x -> vparray[k](x,m,xmesh)[1],x)
end 

# sum over source location or xp contributions over N elements in the mesh  
vp(x,m,xmesh) = sum([vparray[k](x,m,xmesh) for k=1:N])
bflux(x,m,xmesh) = sum([bfluxarray[k](x,m,xmesh) for k=1:N])

# collocate the field on the grid in x - do point matching 
# observe depency in m remains 
bfluxpointsampled(m) = [bflux(xi,m,xmesh) for xi in xmesh]

#average over destination or x contributions 
bfluxaverage(m) = [quadgk(x -> bflux(x,m,xmesh), xmesh[k], xmesh[k+1])[1] for k=1:N]

# compute the Jacobian
# to do: preallocate memory of the Jacobian 
ForwardDiff.jacobian(bfluxpointsampled, m)
ForwardDiff.jacobian(bfluxaverage, m)

5×6 Matrix{Float64}:
  1.56476    1.89875   -0.753994  -0.58825   -0.240827  -0.069234
 -0.261122   1.89875    1.89875   -0.753994  -0.58825   -0.152603
 -0.38242   -0.753994   1.89875    1.89875   -0.753994  -0.38242
 -0.152603  -0.58825   -0.753994   1.89875    1.89875   -0.261122
 -0.069234  -0.240827  -0.58825   -0.753994   1.89875    1.56476

### Section 4.1: Fourth Proof of Concept
Extension from line (bar) to rectangle (plate with zero thickness) assuming in-plane magnetization (i.e. $\vec{M} = (M_x(x,y), M_y(x,y), 0)$) and thus ($\vec{A} = (0, 0, A_z(x,y))$ and $\vec{B} = (B_x(x,y), B_y(x,y), 0)$). 

1. in case of element-wise constant magnetization: asssume both $M_x(x,y)$ and $M_y(x,y)$ to be constant on each rectangular element in the mesh; 
2. in case of linear magnetization: assume both $M_x(x,y)$ and $M_y(x,y)$ to be expanded in a tensor product basis of linear basis element (aka Q1 finite elements, need to include a reference here); 

 <b>Potential Gains</b> We perceive the following gains 
1. allow automatic differentiation to perform half the work. We can thus focus on the remaining part of the work. (do we need to pay a price (toll) later? We need to ensure that in performing the integral computation over xprime, the result remains differentiable wrt x); 
2. preserve order of computations as the math reads, i.e., first integration of the source domain, then differentiate in the destination variables. Code thus easier to read and maintain?;
3. avoid differentiating the integral, thus avoid increasing the singularity of the integrand? 
4. avoid splitting integral as a sum of multiple integrals (and thus simplify the overall bookkeeping); 
5. avoid replacing volume integrals $dx'\,dy'\,dz'$ over elements $e_k$ in the finite element mesh as sum of integrsals over faces of $e_k$ (how to compute singular integrals over tetrahedra?); 
6. allow for more general basis functions; 
7. extend to non-linear constitutive relations (as similar ideas apply);   
8. show-case distinct use of multiple dispatch allowing to compose two Julia packages (cfr. earlier case of combined used of interval arithmetic and ODE solver by Chris Raukaucas);  

<b>Assumptions Made</b> We assume (falsily?) that 
1. the eventual computational overhead of adaptive quadrature (for singular intregrals on tetrahedra) can be amortized by thread-parallel computations (as shown earlier by the heroin in this project);
2. residual weighting of the MoM equations (weak or variational formulations) occurs using Dirac delta (pulse) functions (distribution). Residual weighting thus reduces to point matching;
3. adaptive quadrature carries over from 1D (only x) to 2D (xy) and 3D (xyz); 

<b>One-Dimensional Case</b> <b>What is the good kernel to use?</b> In one spatial dimension ($x$-only), the vector valued problem reduces to a scalar problem. Given the magnitization vector $\vec{M}(\vec{x}) = (0,0,M_z(x))$, we wish to compute (confusing noation: rectify by specifying what $\Omega$ and $\Omega_m$ are)
1. the magnetic vector potential $\vec{A}(\vec{x}) = (0,A_y(x),0)$ where 
$A_y(x) = \frac{\mu_0}{4 \pi} \int_a^b \frac{M_z(x') \, (x-x')}{|x-x'|^3} \, dx'$ using quadrature implemented in QuadGK.jl; 
2. the magnetic flux $\vec{B}(\vec{x}) = (0,0,B_z(x))$ where $B_z(x) = - \frac{\partial A_y(x)}{\partial x}$ using automatic differentiation implemented in ForwardDiff.jl;  

<b>Spatial Discretization of the One-Dimensional Case</b> Introduce computational mesh $\Omega^h$ of $N$ elements $e_k = [x_k, x_{k+1}]$ and $N+1$ nodes $x_i$ on the domain of computation $\Omega$. On $e_k$ linear nodal shape function $\phi_i(x)$ can be defined. Express numerical approximation to $M_z(x)$ as $M^h_z(x) = \sum_i m_i \, \phi_i(x)$. Denote $\vec{m}$ the (N+1)-dimensional vector (one per node) of the mesh. Replacing $M_z(x)$ as $M^h_z(x)$ in the expression for $A_y(x)$ yields $A^h_y(x)$. The evaluation of $A^h_y(x)$ in the nodes $x_i$ yields a (N+1)-dimensional vector $\vec{a}$. This vector depends on $\vec{m}$. We thus write $\vec{a}(\vec{m})$. Given its importance in later discussions, we emphasize the point by formulating $\vec{a}$ as a mapping from $R^{N+1}$ to $R^{N+1}$ mapping $\vec{m}$ to $\vec{a}(\vec{m})$. A similar discourse holds for $B_z(x)$ by first differentiating $A^h_y(x)$ wrt to $x$ and then point-evaluation (order obviously matters here). We again obtain a mapping from $R^{N+1}$ to $R^{N+1}$ mapping $\vec{m}$ this time to $\vec{b}[\vec{a}(\vec{m})]$. The Jacobian of this mapping wrt $\vec{m}$ is the much desired $(N+1)-by-(N+1)$ MoM matrix.   

<b>Resulting Algorithm for a Forward Computation</b>

1. Generate mesh of $\Omega$
2. Generate set basis functions 
3. (to be extended) 

<b>Two-Dimensional Perpendicular Magnetization Case</b> To be discussed.

<b>Two-Dimensional In-Plane Magnetization Case</b> To be discussed.

<b>Three-Dimensional Mesh Generation</b> To be discussed.
1. integrate 6D integral with singular kernel using Curbature.jl; allow adaptive integration to do the heavy work. 
2. as above with task parallel implementation; 
3. as above with analytical computation of the inner 3D integral; 

<b>Spatial Discretization of the Three-Dimensional Case</b> To be discussed. 

<b>Reference on Quadrature in Julia</b>
1. [QuadGK.jl](https://juliamath.github.io/QuadGK.jl/stable/) for adaptive quadrature (requires context on quadrature (adaptive and non-adaptive) on meshes in Julia - ask our heroin for first draft); 
2. [Integrals.jl](https://docs.sciml.ai/Integrals/stable/) a unified interface for the numerical approximation of integrals (quadrature) in Julia; 
3. [Cubature of singular integrand](https://discourse.julialang.org/t/cubature-of-singular-integrand/113003) 
4. [ForwardDiff.jl](https://juliadiff.org/ForwardDiff.jl/stable/) for automatic differentiation (requires context on automatic (forward mode and backward mode) differentiation in Julia - ask our heroin for first draft); 
5. [How-do-i-choose-between-quadgk-and-cubature-when-i-do-singular-integral-of-a-complex-valued-function](https://discourse.julialang.org/t/how-do-i-choose-between-quadgk-and-cubature-when-i-do-singular-integral-of-a-complex-valued-function/94198/1) (including link to interesting paper). The [paper](https://math.mit.edu/~stevenj/papers/ReidWhJo14.pdf).  
6. [Cubature on non-rectangular domains](https://discourse.julialang.org/t/2d-integration-over-non-rectangular-domain-using-cubature/2991/6); Quote: In general, a 2d cubature code can be more efficient than nested 1d quadratures, assuming you are using adaptive quadrature/cubature. This is because the inner 1d integral, if it is performed as an independent 1d adaptive quadrature, can waste a lot of integrand evaluations trying to refine the 1d integral to high relative accuracy even if its overall contribution to the integral is small. End-quote. Quote: to integrate f(x,y) for x=0…t and y=0…x (i.e. a triangle), just do the change of variables y=ux and integrate f(x,u*x)*x for x=0…t , u=0…1 (note the Jacobian factor *x multiplying f). End-quote. 

<b>Reference to Automatic Differentiation in Julia</b>
1. [ForwardDiff (Forward Mode AD)](https://juliadiff.org/ForwardDiff.jl/stable/) for automatic differentiation (requires context on automatic (forward mode and backward mode) differentiation in Julia - ask our heroin for first draft); 
2. [When Zygote (Reverse Mode AD) Meets Quadgk](https://discourse.julialang.org/t/when-zygote-jl-meets-quadgk-jl-mutating-arrays-is-not-supported-called-setindex-vector-quadgk-segment/84042) 
3. [AutodiffTutorial by G. Dalle](https://gdalle.github.io/AutodiffTutorial/)
4. [How-do-i-choose-between-quadgk-and-cubature-when-i-do-singular-integral-of-a-complex-valued-function](https://discourse.julialang.org/t/how-do-i-choose-between-quadgk-and-cubature-when-i-do-singular-integral-of-a-complex-valued-function/94198/1) (including [link](https://math.mit.edu/~stevenj/papers/ReidWhJo14.pdf) to paper on Taylor-Duffy method for analytically handling the singularity of the integrand in the scuff-em library).  
5. [Tutorials on differentiating_integrals](https://docs.sciml.ai/Integrals/stable/tutorials/differentiating_integrals/)
6. [forwarddiff-hessian-with-staticarrays-unexpected-allocations-and-performance](https://discourse.julialang.org/t/forwarddiff-hessian-with-staticarrays-unexpected-allocations-and-performance/114082) 

<b>References to Related Tools in Julia</b>
1. [Monte-Carlo Integration](https://numericaleft.github.io/NumericalEFT.jl/stable/readme/MCIntegration/): is this a good test for analytically handling the singularity? 


The magic resides in the multiple dispatch that allows to chain QuadGK.jl and ForwardDiff.jl.

<b>Related Packages</b> 

- [Meshes.jl](https://juliageometry.github.io/MeshesDocs/stable/) and related Zulip channel; 
- [MeshIntegrals.jl](https://github.com/mikeingold/MeshIntegrals.jl) 
- [SingularIntegralEquations.jl](https://github.com/JuliaApproximation/SingularIntegralEquations.jl)



<b>How to Procede from Here?</b>
1. replace implementation that employ an array of function to code the basis function by common loop over elements; 
2. extend to 2D

<b>References</b> (ask our heroin to expand) 
1. Book Roman Szewczyk (Springer LNEE 491 2018)
2. Caorsi, Moreno and Sidoti, 1993: test case of block profile in Figure 4; 

<b>Notation</b> We use the following notation: 
- xp for xprime; 
- vpdens for vector potential density;
- vp for vector potential; 
- $k$ and $ell$: outer loop over elements (k) and DOF per element (ell); 
- $m$ and $n$: outer loop over elements (m) and DOF per element (n); 
- i: loop over nodes; 

## Section 2.1: Singular Integrals (sandbox)

The thing is that as long as r∉∂V, the integrand is a differentiable function, so standard refinement eventually works fine.

But to make it even better, if Fi are the faces of the cube, for each i, you look for the r∗i∈Fi that is closest to r, this is where the almost singularity will be. Then divide the face in 4 sections, where r∗ is the point of intersection of this 4 sections. Now you have 4 integrals that they have almost-singularities in one corner.

Replacing the volume integral over a cell by a set of surface integrals over the faces to allow a better grip on the singularity. Operations on 3D mesh thus required: 
1. retrieve faces belonging to a cell;
2. subdivide a face;
3. compute distance to nearest vertex; 

## Section 3.1: Linear Basis Functions (in progress)

basisfct (generic function with 1 method)

magnetization (generic function with 2 methods)

#21 (generic function with 1 method)

## Section 2: QuadQK, HCurbature, ForwardDiff and Their Combined Use 

### Section 1.2: Examples of Using QuadQK for Scalar Function  
Small example of QuadQK. Singular integrals on purpose avoided (see next Section). Outputs integral value and error. Morer later (amount of function evaluations). 

In [ ]:
# define single input integrand
integrand(x) = x^2 

I = quadgk(x -> integrand(x), 0, 1)

### Section 2.2: Examples of Using HCubature for Scalar Function 

In [ ]:
# define single input integrand
integrand(x) = x[1]^2 # observe that typing x^2 here does not work 

I = hcubature(x->integrand(x), (0.,), (1.,)) 

### Section 3.2: Examples of Using HCubature for Vector-Valued Function
Example found [here](https://stackoverflow.com/questions/77446561/integrating-with-hcubature-in-julia). 

In [ ]:
f(x,y,z) = x+y+z;

### Section 4.2: Nested QuadGK vs. HCubature

In [ ]:
# integration over a unit square 
hcubature(r->f(r[1],r[2],0), (0,0), (1,1))

In [ ]:
h(x,y,z) = x+y 

In [ ]:
H(y) = quadgk(x -> h(x,y,0),0,1)[1]

In [ ]:
H(.5)

In [ ]:
integral = quadgk(y ->H(y),0,1)[1]

In [ ]:
# integration over a unit square 
hcubature(r->h(r[1],r[2],0), (0,0), (1,1))[1]

### Section 5.2: Examples of ForwardDiff for Scalar Case 
Small examples of ForwardDiff. More later (Jacobian and preallocating memory for dense Jacobian). 

In [ ]:
ForwardDiff.derivative(x -> x^3, 1)

### Section 6.2: Examples of ForwardDiff for Vector-Valued Case
See this [notebook](./fem_matrix_assembly_using_ad.ipynb) for other examples. 

In [ ]:
function residual(m)
    r = similar(m)
    r[1] = m[1]
    for i=2:length(m)-1
        r[i] = -m[i-1] + 2*m[i] - m[i+1] 
    end 
    r[end] = m[end]
    return r
end

In [ ]:
m = ones(5)
r = residual(m)
display(r)

In [ ]:
ForwardDiff.jacobian(residual, m) 

### Section 7.2: Examples Involving Generic Functions 

In [ ]:
g = m->m.^4

In [ ]:
m = ones(5)
ForwardDiff.jacobian(g, m)

In [ ]:
g = (m,n) -> m.*n # vector values output 
m = ones(5)
n = 2*ones(5)
h = m->g(m,n)
ForwardDiff.jacobian(h, m)

### Section 8.2: Entire workflow on Complete Example 

In [ ]:
# define function in three variables 
# x: spatial variables: location of observation points 
# m: expansion coefficient of magnetization in basis functions 
# p: parameters: in practise the mesh parameters 
g = (x,m,p) -> (x.*m.^5).*p # vector values output 
# Step (1/2): fix x - do point matching
# after point matching we are left with a function depending on m and p 
# this function is named h(m,p)
x = ones(5)
h = (m,p)->g(x,m,p)
# Step (2/2): take the gradient wrt m to 
hgrad(m,p) = ForwardDiff.jacobian(m->h(m,p), m)

In [ ]:
m = ones(5)
p = ones(5)
hgrad(m,p) 

### Section 9.2: Example of Derivate of an Integral 
Small example of derivative of an integral. More later. 

In [ ]:
# define two input integrand (here p stands for prime)
integrand(x,xp) = x[1] - xp[1]

# compute integral by quadrature over second input - results in u(x) = x - 0.5
# u(x) = quadgk(xp -> integrand(x,xp), 0, 1)
u(x) = hcubature(xp -> integrand(x,xp), (0,), (1,))

# compute derivative - should result in du/dx = 1
ForwardDiff.derivative(x -> u(x)[1], 1)

## Section 3: QuadQK for Singular Integrals  
The following example was constructed to illustrate difficulties with singular kernels. See [QuadQK.jl](https://juliamath.github.io/QuadGK.jl/stable/) for treating singular integrals.  

### Section 1.3: Converging Improper Integral 

In [7]:
# define two input integrand
integrand(x,xp) = (x-xp)/abs(x - xp)^1.5

# compute integral by quadrature over second input
u(x) = quadgk(xp -> integrand(x,xp), 0, 1)

# compute derivative - should result in du/dx = 1
ForwardDiff.derivative(x -> u(x)[1], 1)

-9.514789506324105e7

Please <b>observe</b> that in the previous cell the function $u(x)$ is <b>never</b> evaluated. This is particulary interesting, as we can evaluate $u(x)$ for $x$ such that $0 < x < 1$ (i.e. $x$ not coinciding with bounds of domain of integration, and integrand singular in the interior of the interval) (quadrature fails) and $x=1$ (i.e. $x$ coinciding with upper integration bound) (quadrature works).  

In [10]:
u(0.)

(-1.9999999845983918, 2.3762512035610803e-8)

In [11]:
u(1)

(1.9999999875459993, 2.6379133460101236e-8)

### Section 2.3: Diverging Improper Integral 
This subsection shows that the magic that the function quadqk() can perform remains limited. In case that the singularity is to strong, quadqk() is not able to resolve it, even in case that the point of singularity matches with the bound on the integration domain. Possibly looking into singularity extraction becomes of interest here. 

In [35]:
?quadgk

search: quadgk quadgk! QuadGK quadgk_print quadgk_count



```
quadgk(f, a,b,c...; rtol=sqrt(eps), atol=0, maxevals=10^7, order=7, norm=norm, segbuf=nothing)
```

Numerically integrate the function `f(x)` from `a` to `b`, and optionally over additional intervals `b` to `c` and so on. Keyword options include a relative error tolerance `rtol` (if `atol==0`, defaults to `sqrt(eps)` in the precision of the endpoints), an absolute error tolerance `atol` (defaults to 0), a maximum number of function evaluations `maxevals` (defaults to `10^7`), and the `order` of the integration rule (defaults to 7).

Returns a pair `(I,E)` of the estimated integral `I` and an estimated upper bound on the absolute error `E`. If `maxevals` is not exceeded then `E <= max(atol, rtol*norm(I))` will hold. (Note that it is useful to specify a positive `atol` in cases where `norm(I)` may be zero.)

The endpoints `a` et cetera can also be complex (in which case the integral is performed over straight-line segments in the complex plane). If the endpoints are `BigFloat`, then the integration will be performed in `BigFloat` precision as well.

!!! note
    It is advisable to increase the integration `order` in rough proportion to the precision, for smooth integrands.


More generally, the precision is set by the precision of the integration endpoints (promoted to floating-point types).

The integrand `f(x)` can return any numeric scalar, vector, or matrix type, or in fact any type supporting `+`, `-`, multiplication by real values, and a `norm` (i.e., any normed vector space). Alternatively, a different norm can be specified by passing a `norm`-like function as the `norm` keyword argument (which defaults to `norm`).

!!! note
    Only one-dimensional integrals are provided by this function. For multi-dimensional integration (cubature), there are many different algorithms (often much better than simple nested 1d integrals) and the optimal choice tends to be very problem-dependent. See the Julia external-package listing for available algorithms for multidimensional integration or other specialized tasks (such as integrals of highly oscillatory or singular functions).


The algorithm is an adaptive Gauss-Kronrod integration technique: the integral in each interval is estimated using a Kronrod rule (`2*order+1` points) and the error is estimated using an embedded Gauss rule (`order` points). The interval with the largest error is then subdivided into two intervals and the process is repeated until the desired error tolerance is achieved.

These quadrature rules work best for smooth functions within each interval, so if your function has a known discontinuity or other singularity, it is best to subdivide your interval to put the singularity at an endpoint. For example, if `f` has a discontinuity at `x=0.7` and you want to integrate from 0 to 1, you should use `quadgk(f, 0,0.7,1)` to subdivide the interval at the point of discontinuity. The integrand is never evaluated exactly at the endpoints of the intervals, so it is possible to integrate functions that diverge at the endpoints as long as the singularity is integrable (for example, a `log(x)` or `1/sqrt(x)` singularity).

For real-valued endpoints, the starting and/or ending points may be infinite. (A coordinate transformation is performed internally to map the infinite interval to a finite one.)

In normal usage, `quadgk(...)` will allocate a buffer for segments. You can instead pass a preallocated buffer allocated using `alloc_segbuf(...)` as the `segbuf` argument. This buffer can be used across multiple calls to avoid repeated allocation.

---

```
quadgk(f::BatchIntegrand, a,b,c...; kws...)
```

Like [`quadgk`](@ref), but batches evaluation points for an in-place integrand to evaluate simultaneously. In particular, there are two differences from `quadgk`

1. The function `f.f!` should be of the form `f!(y, x) = y .= f.(x)`.  That is, it writes the return values of the integand `f(x)` in-place into its first argument `y`. (The return value of `f!` is ignored.) See [`BatchIntegrand`](@ref) for how to define the integrand.
2. `f.max_batch` changes how the adaptive refinement is done by batching multiple segments together. Choosing `max_batch<=4*order+2` will reproduce the result of non-batched `quadgk`, however if `max_batch=n*(4*order+2)` up to `2n` Kronrod rules will be evaluated together, which can produce slightly different results and sometimes require more integrand evaluations when using relative tolerances.


In [48]:
# define two input integrand
function integrand(x)
    # print the input value for xp allowing to understand 
    # in which locations adaptive integration evaluates the integrand 
    println(x)
    return 1/x
end 

# compute integral by quadrature over second input
int = quadgk(x -> integrand(x), 0, 1;rtol=5e-1,atol=10,order=5)

0.5
0.36018479341910836
0.6398152065808916
0.04691007703066802
0.9530899229693319
0.007957319952578756
0.9920426800474212
0.23076534494715845
0.7692346550528415
0.12291663671457537
0.8770833632854247


(6.406074888327447, 1.839408221660781)

In [46]:
u(0)

0.5
0.36018479341910836
0.6398152065808916
0.04691007703066802
0.9530899229693319
0.007957319952578756
0.9920426800474212
0.23076534494715845
0.7692346550528415
0.12291663671457537
0.8770833632854247
0.25
0.18009239670955418
0.3199076032904458
0.02345503851533401
0.47654496148466596
0.003978659976289378
0.4960213400237106
0.11538267247357922
0.38461732752642075
0.06145831835728768
0.43854168164271234
0.75
0.6800923967095542
0.8199076032904458
0.523455038515334
0.976544961484666
0.5039786599762894
0.9960213400237106
0.6153826724735793
0.8846173275264207
0.5614583183572877
0.9385416816427123
0.125
0.09004619835477709
0.1599538016452229
0.011727519257667005
0.23827248074233298
0.001989329988144689
0.2480106700118553
0.05769133623678961
0.19230866376321037
0.03072915917864384
0.21927084082135617
0.375
0.3400461983547771
0.4099538016452229
0.261727519257667
0.488272480742333
0.2519893299881447
0.4980106700118553
0.3076913362367896
0.4423086637632104
0.28072915917864383
0.46927084082135617
0

1.1826070309250608e-7
2.7509372824091726e-8
9.169991672668952e-8
1.4652804936716004e-8
1.0455648461406525e-7
1.7881393432617188e-7
1.6214666288126808e-7
1.9548120577107567e-7
1.248014065063796e-7
2.3282646214596414e-7
1.2015787600905643e-7
2.3746999264328732e-7
1.4671866237487298e-7
2.1090920627747077e-7
1.3386209448749725e-7
2.237657741648465e-7
2.9802322387695312e-8
2.146868666524341e-8
3.8135958110147216e-8
2.7960584777991783e-9
5.6808586297591443e-8
4.742932291375849e-10
5.913035154625304e-8
1.3754686412045863e-8
4.584995836334476e-8
7.326402468358002e-9
5.2278242307032626e-8
8.940696716308594e-8
8.107333144063404e-8
9.774060288553783e-8
6.24007032531898e-8
1.1641323107298207e-7
6.007893800452822e-8
1.1873499632164366e-7
7.335933118743649e-8
1.0545460313873538e-7
6.693104724374862e-8
1.1188288708242325e-7
1.4901161193847656e-8
1.0734343332621704e-8
1.9067979055073608e-8
1.3980292388995891e-9
2.8404293148795722e-8
2.3714661456879247e-10
2.956517577312652e-8
6.8773432060229315e-9
2.2

1.3117491161390174e-14
4.372592769941784e-14
6.98700186572838e-15
4.985641699507964e-14
8.526512829121202e-14
7.7317553940424e-14
9.321270264200004e-14
5.950994801825505e-14
1.11020308564169e-13
5.729574013188192e-14
1.1323451645054212e-13
6.996091002219819e-14
1.0056934656022585e-13
6.383042072653639e-14
1.0669983585588765e-13
1.4210854715202004e-14
1.0237067539807991e-14
1.8184641890596016e-14
1.3332645787235156e-15
2.708844485168049e-14
2.261606355369496e-16
2.8195548794867058e-14
6.558745580695087e-15
2.186296384970892e-14
3.49350093286419e-15
2.492820849753982e-14
4.263256414560601e-14
3.8658776970212e-14
4.660635132100002e-14
2.9754974009127525e-14
5.55101542820845e-14
2.864787006594096e-14
5.661725822527106e-14
3.4980455011099096e-14
5.0284673280112926e-14
3.1915210363268196e-14
5.3349917927943826e-14
7.105427357601002e-15
5.118533769903996e-15
9.092320945298008e-15
6.666322893617578e-16
1.3544222425840245e-14
1.130803177684748e-16
1.4097774397433529e-14
3.2793727903475435e-15
1

8.271657568526938e-22
4.268865297129893e-22
8.436628911684612e-22
5.212493987545236e-22
7.493000221269269e-22
4.755736941586166e-22
7.949757267228339e-22
1.0587911840678754e-22
7.627209678149217e-23
1.3548614003208292e-22
9.933595200803247e-24
2.0182464161277183e-22
1.6850280429195584e-24
2.1007320877065553e-22
4.886646256368672e-23
1.6289177424988835e-22
2.6028610265733228e-23
1.8572962654784187e-22
3.1763735522036263e-22
2.880303335950673e-22
3.47244376845658e-22
2.2169183201437834e-22
4.135828784263469e-22
2.1344326485649467e-22
4.218314455842306e-22
2.606246993772618e-22
3.7465001106346344e-22
2.377868470793083e-22
3.9748786336141695e-22
5.293955920339377e-23
3.8136048390746085e-23
6.774307001604146e-23
4.966797600401624e-24
1.0091232080638591e-22
8.425140214597792e-25
1.0503660438532776e-22
2.443323128184336e-23
8.144588712494418e-23
1.3014305132866614e-23
9.286481327392093e-23
1.5881867761018131e-22
1.4401516679753364e-22
1.73622188422829e-22
1.1084591600718917e-22
2.067914392131

1.771255746344012e-28
3.0292258760486853e-28
2.7468713149553993e-28
3.3115804371419714e-28
2.1142180634916148e-28
3.944233688605756e-28
2.035553597035357e-28
4.0228981550620135e-28
2.4855108201719457e-28
3.572940931925425e-28
2.2677120883875685e-28
3.790739663709802e-28
5.048709793414476e-29
3.636936987948044e-29
6.460482598880907e-29
4.736707306291221e-30
9.623748856199828e-29
8.034839834783355e-31
1.0017071188481118e-28
2.330134514030777e-29
7.767285072798174e-29
1.2411408551088919e-29
8.85627873172006e-29
1.5146129380243427e-28
1.3734356574776996e-28
1.6557902185709857e-28
1.0571090317458074e-28
1.972116844302878e-28
1.0177767985176786e-28
2.0114490775310068e-28
1.2427554100859729e-28
1.7864704659627125e-28
1.1338560441937842e-28
1.895369831854901e-28
2.524354896707238e-29
1.818468493974022e-29
3.2302412994404534e-29
2.3683536531456106e-30
4.811874428099914e-29
4.017419917391678e-31
5.008535594240559e-29
1.1650672570153885e-29
3.883642536399087e-29
6.2057042755444594e-30
4.428139365

...

1.6882086423921104e-84
1.0778059825019548e-84
2.0107285711788625e-84
1.0377036704363503e-84
2.050830883244467e-84
1.2670870984473965e-84
1.821447455233421e-84
1.1560556111320051e-84
1.9324789425488122e-84
2.5737787947340145e-85
1.8540719669755054e-85
3.2934856224925235e-85
2.41472323041745e-86
4.9060852664262836e-85
4.096076271372215e-87
5.106596826754307e-85
1.1878779027689531e-85
3.9596796866990755e-85
6.32720466191997e-86
4.514837123276032e-85
7.721336384202043e-85
7.001629556443535e-85
8.441043211960552e-85
5.389029912509774e-85
1.0053642855894313e-84
5.188518352181752e-85
1.0254154416222335e-84
6.335435492236982e-85
9.107237276167104e-85
5.780278055660026e-85
9.662394712744061e-85
1.2868893973670072e-85
9.270359834877527e-86
1.6467428112462618e-85
1.207361615208725e-86
2.4530426332131418e-85
2.0480381356861077e-87
2.5532984133771534e-85
5.9393895138447656e-86
1.9798398433495378e-85
3.163602330959985e-86
2.257418561638016e-85
3.8606681921010217e-85
3.5008147782217674e-85
4.22052160

6.0340925807189655e-92
4.3056842072258303e-91
7.36364019794659e-91
6.677274280971084e-91
8.050006114922096e-91
5.139379417905592e-91
9.587900977987587e-91
4.948156692678215e-91
9.779123703214965e-91
6.041942112194998e-91
8.685338283698182e-91
5.512502723369623e-91
9.214777672523557e-91
1.2272733663244316e-91
8.840904078366782e-92
1.570456324812185e-91
1.1514297630393267e-92
2.3394037563449305e-91
1.9531613690243794e-93
2.4350151189586195e-91
5.6642432344863563e-92
1.8881224092002275e-91
3.0170462903594828e-92
2.1528421036129151e-91
3.681820098973295e-91
3.338637140485542e-91
4.025003057461048e-91
2.569689708952796e-91
4.793950488993794e-91
2.4740783463391073e-91
4.8895618516074825e-91
3.020971056097499e-91
4.342669141849091e-91
2.7562513616848114e-91
4.6073888362617784e-91
6.136366831622158e-92
4.420452039183391e-92
7.852281624060925e-92
5.757148815196634e-93
1.1697018781724652e-91
9.765806845121897e-94
1.2175075594793097e-91
2.8321216172431782e-92
9.440612046001138e-92
1.5085231451797

1.8626798334354204e-99
2.322211379011745e-97
5.401843294607502e-98
1.8006538478853488e-97
2.8772795585245922e-98
2.05311022149364e-97
3.511257266019149e-97
3.1839724926810663e-97
3.838542039357231e-97
2.4506470765617333e-97
4.571867455476564e-97
2.3594649756804536e-97
4.663049556357844e-97
2.8810225068068495e-97
4.141492025231448e-97
2.628566133198558e-97
4.393948398839739e-97
5.852095443365248e-98
4.2156715766748344e-98
7.488519310055661e-98
5.490444960781702e-99
1.1155146390652325e-97
9.313399167177102e-100
1.1611056895058725e-97
2.700921647303751e-98
9.003269239426744e-98
1.4386397792622961e-98
1.02655511074682e-97
1.7556286330095744e-97
1.5919862463405331e-97
1.9192710196786156e-97
1.2253235382808667e-97
2.285933727738282e-97
1.1797324878402268e-97
2.331524778178922e-97
1.4405112534034248e-97
2.070746012615724e-97
1.314283066599279e-97
2.1969741994198696e-97
2.926047721682624e-98
2.1078357883374172e-98
3.744259655027831e-98
2.745222480390851e-99
5.5775731953261625e-98
4.65669958358

3.34859587289729e-103
3.0364727904139197e-103
3.66071895538066e-103
2.337119175492986e-103
4.360072570301594e-103
2.250161147766546e-103
4.447030598028034e-103
2.747557169730043e-103
3.949634576064537e-103
2.506796010206755e-103
4.190395735587825e-103
5.5809931214954833e-104
4.020377709078631e-104
7.141608533912336e-104
5.2360963447396295e-105
1.0638376608517003e-103
8.88194958417616e-106
1.1073166747149205e-103
2.5757996056592475e-104
8.586186637331718e-104
1.3719938080428086e-104
9.789992434948159e-104
1.674297936448645e-103
1.5182363952069599e-103
1.83035947769033e-103
1.168559587746493e-103
2.180036285150797e-103
1.125080573883273e-103
2.223515299014017e-103
1.3737785848650215e-103
1.9748172880322685e-103
1.2533980051033775e-103
2.0951978677939125e-103
2.7904965607477417e-104
2.0101888545393154e-104
3.570804266956168e-104
2.6180481723698148e-105
5.3191883042585015e-104
4.44097479208808e-106
5.5365833735746025e-104
1.2878998028296238e-104
4.293093318665859e-104
6.859969040214043e-10

LoadError: DomainError with 5.450188595210433e-107:
integrand produced Inf in the interval (0.0, 1.0900377190420866e-106)

## Section 4: Generation of Basis Functions on 1D Mesh 

### Section 1.4: Generate Family of 1D Basis Hat Functions

In [ ]:
# generate basis function centered on node i 
function basisfct(x, xmesh, i)
    Np1 = length(xmesh)
    left = 0.
    right = 0.  
    if (i==1)
        right_domain = (x>=xmesh[i])*(x<=xmesh[i+1])
        right_value = (x-xmesh[i+1])/(xmesh[i]-xmesh[i+1])
        right = right_value*right_domain        
    end 
    if ((i>1)&&(i<Np1))
        left_domain = (x>=xmesh[i-1])*(x<=xmesh[i])
        left_value = (xmesh[i-1]-x)/(xmesh[i-1]-xmesh[i])
        left = left_value*left_domain        
        right_domain = (x>xmesh[i])*(x<=xmesh[i+1])
        right_value = (x-xmesh[i+1])/(xmesh[i]-xmesh[i+1])
        right = right_value*right_domain        
    end 
    if (i==Np1)
        left_domain = (x>=xmesh[i-1])*(x<=xmesh[i])
        left_value = (xmesh[i-1]-x)/(xmesh[i-1]-xmesh[i])
        left = left_value*left_domain
    end 
    result = left+right
    return result 
end 

# generate the mesh with N elements and Np1 nodes 
N = 5; h = 1/N; Np1 = N+1; 
xmesh = Vector(0:h:1)

# generate plots 
xsampled = Vector(0:h/10:1);
p1 = plot(xsampled,basisfct.(xsampled,Ref(xmesh),6))
p2 = plot(xsampled,basisfct.(xsampled,Ref(xmesh),1) + (1)*basisfct.(xsampled,Ref(xmesh),4))
plot(p1,p2,layout=(1,2))

### Section 2.5: Expand Magnetization in Basis of Hat Functions

In [ ]:
function magnetization(x,m,xmesh)
    Np1 = length(xmesh)
    result = 0. 
    for i=1:Np1 
        result += m[i]*basisfct(x,xmesh,i)
    end 
    return result  
end 

# generate the mesh with N elements and Np1 nodes 
N = 5; h = 1/N; Np1 = N+1; 
xmesh = Vector(0:h:1)
xsampled = Vector(0:h/2:1)

# generate plots 
m = ones(length(xmesh)) 
m[2] = 2; m[4] = -3; 
magnsampled = magnetization.(xsampled,Ref(m),Ref(xmesh))
plot(xsampled,magnsampled)
#display(magnsampled)

## Section 5: Explicit (or Direct) Construction - Assume Magnetic Sources Known - Compute Magnetic Vector Potential A_y and Magnetic Flux B_z 
Here we assume that the magnetization is a given block profile. We compute the vector potential component and the magnetic flux component. Results in this section can be verified by solving the one-dimensional Poisson equation for the scalar magnetic vector potential ($\phi_m$ such that $\vec{H} = \nabla \phi_m$). This can easily be accomplished using a shooting method for the boundary value problem. Such a shooting method is implemented in DifferentialEquations.jl. See the EE4375 course for example. 

### Section 1.5: Assume $M_z(x) = 1$ between $x=0.4$ and $x=0.6$: First Alternative 
This construction fails as quadgk is unable to treat the singular kernel without guidance. 

In [ ]:
# define two input integrand
a = 0.4; b = 0.6; 
magnetization(x) = ((x>a)*(x<b))
kernel(x,xp) = (x-xp)/abs(x - xp)^3
vpdens(x,xp) = magnetization(xp)*kernel(x,xp)

# compute integral by quadrature over second input 
# observe the integration bounds 
vp(x) = quadgk(xp -> vpdens(x,xp), 0, 1)

# compute flux as x-derivative of potential  
bflux(x) = ForwardDiff.derivative(x -> vp(x)[1])

# generate plot 
N = 100; h = 1/N; 
xs = Vector(0:h:1)
ms = magnetization.(xs)
vps = vp.(xs)
bfluxs = bflux.(xs)

p1 = plot(xs, ms)
p2 = plot(xs, vps)
p3 = plot(xs, bfluxs)
plot(p1,p2,p3,layout=(1,3))

### Section 2.5: Assume $M_z(x) = 1$ between $x=0.4$ and $x=0.6$: Second Alternative
Remark: results in this section are <b>very</b> sensitive to the kernel definition. Here we use as kernel definition 
$$ K(x,x') = \frac{x-x'}{|x-x'|^{1.5}} $$
(observe the exponent 1.5 in the denominatior) for the lousy argumemt that anything else does not work (at least yet). 

Guide quadgk in performing integration of singular kernel by divide-and-conquer. We apply a mesh on the interval $\Omega = (0,1)$, perform integration on each element on the mesh and evaluate the integral on the nodes of the mesh. In this way we ensure that <b>singularity of integrand is always on the boundary of the integration domain</b>. 

We proceed in two steps. In the first step, we evaluate the vector potential component as a sum of contributions per element 

$A_y(x) = \frac{\mu_0}{4 \pi} \int_0^1 \frac{M_z(x') \, (x-x')}{|x-x'|^{1.5}} \, dx' 
        = \frac{\mu_0}{4 \pi} \sum_{k=1}^{N} \int_{x_k}^{x_{k+1}} \frac{M_z(x') \, (x-x')}{|x-x'|^{1.5}} \, dx' \, . $
        
This component is a function of $x$. In the second step, we evaluate $A_y(x)$ in the nodes of the mesh. 

In [ ]:
# generate the mesh with N elements (intervals) and N+1 nodes 
N = 99; h = 1/N; Np1 = N+1; 
# N = 1000; h = 1/N; Np1 = N+1; 
xmesh = Vector(0:h:1)

# define two input integrand
a = 0.4; b = 0.6; 
#a = 0.; b = 1.; 
magnetization(x) = ((x>=a)*(x<=b)) 
kernel(x,xp) = (x-xp)/abs(x - xp)^1.5 # works and yield nice plot 
vpdens(x,xp) = magnetization(xp)*kernel(x,xp)
 
vparray = Array{Function}(undef, N)
# compute integral by quadrature over second input over all elements [x_k, x_{k+1}]
# observe syntax used to define an array of functions 
for k=1:N
    vparray[k] = x -> quadgk(xp -> vpdens(x,xp), xmesh[k], xmesh[k+1])[1]
end 

bfluxarray = Array{Function}(undef, N)
# compute bflux as x-derivative over all elements of vparray 
# observe syntax used to define an array of functions 
for k=1:N
  bfluxarray[k] = x -> ForwardDiff.derivative(x -> vparray[k](x)[1],x)
end 

In [ ]:
plot(xmesh, magnetization.(xmesh),xaxis="x (m)",yaxis="M_z(x)",label="Prescribed Magnetization")

In [ ]:
plot(xmesh,[vparray[k].(xmesh) for k=1:N],xaxis="x (m)",yaxis="A_y(x)",title="vp per element")

In [ ]:
# evaluate vector potential as sum of contribution over N elements 
vparrayvec = sum([vparray[k].(xmesh) for k=1:N],dims=1)
plot(xmesh,vparrayvec)

In [ ]:
bfluxarrayvec = sum([bfluxarray[k].(xmesh) for k=1:N],dims=1)
plot(xmesh,bfluxarrayvec,xaxis="x (m)",yaxis="B_z(x)",title="magnetic flux component")

### Section 3.5: Magnitization Expanded in a Basis 

In [ ]:
function magnetization(x,m,xmesh)
    Np1 = length(xmesh)
    result = 0. 
    for i=1:Np1 
        result += m[i]*basisfct(x,xmesh,i)
    end 
    return result  
end 

In [ ]:
# generate the mesh  with N elements (intervals) and N+1 nodes
N = 100; h = 1/N; Np1 = N+1; 
xmesh = Vector(0:h:1)

# set the magnetization coefficients  
m = zeros(length(xmesh)) 
for i=div(N,4):3*div(N,4) m[i] = 1. end 

# define two input integrand
kernel(x,xp) = (x-xp)/abs(x - xp)^1.5
vpdens(x,xp,m,xmesh) = magnetization(xp,m,xmesh)*kernel(x,xp)

# intermediate auxilary 
#vptest = (x,m,xmesh) -> quadgk(xp -> vpdens(x,xp,m,xmesh), xmesh[1], xmesh[2])[1]

vparray = Array{Function}(undef, N)
# compute integral by quadrature over second input over all elements [x_k, x_{k+1}]
# observe syntax used to define an array of functions 
for k=1:N
   vparray[k] = (x,m,xmesh) -> quadgk(xp -> vpdens(x,xp,m,xmesh), xmesh[k], xmesh[k+1])[1]
end 

bfluxarray = Array{Function}(undef, N)
# compute bflux as x-derivative over all elements of vparray 
# observe syntax used to define an array of functions 
for k=1:N
  bfluxarray[k] = (x,m,xmesh) -> ForwardDiff.derivative(x -> vparray[k](x,m,xmesh)[1],x)
end 

# sum over elements 
vp(x,m,xmesh) = sum([vparray[k](x,m,xmesh) for k=1:N])
bflux(x,m,xmesh) = sum([bfluxarray[k](x,m,xmesh) for k=1:N])

In [ ]:
bflux.(xmesh,Ref(m),Ref(xmesh)) ;

In [ ]:
# fix first argument - define a trace function 
vpsampled = (m,xmesh) -> vptest.(xmesh,Ref(m),Ref(xmesh)) 

In [ ]:
# why ? 
vpsampled.(m,xmesh)

In [ ]:
bfluxsampled = (m,xmesh) -> bflux(xmesh,m,xmesh)

In [ ]:
bfluxsampled.(m,Ref(xmesh))

In [ ]:
bfluxgrad(m,xmesh) = ForwardDiff.jacobian(m->bfluxsampled(m,xmesh), m)

In [ ]:
bfluxgrad(m,xmesh)

In [ ]:
# vparray[1].(xmesh,Ref(m),Ref(xmesh)) # works 
vparray[1].(Ref(xmesh),m,Ref(xmesh)) # fails - due to how QuadQK expects its inputs 

In [ ]:
magnsampled = magnetization.(xsampled,Ref(m),Ref(xmesh))
plot(xsampled,magnsampled,xaxis="x (m)",yaxis="M_z(x)",label="Prescribed Magnetization")

In [ ]:
plot(xmesh,[vparray[k].(xmesh,Ref(m),Ref(xmesh)) for k=1:N],xaxis="x (m)",yaxis="A_y(x)",title="vp per element")

In [ ]:
# evaluate vector potential as sum of contribution over N elements 
vparrayvec = sum([vparray[k].(xmesh,Ref(m),Ref(xmesh)) for k=1:N],dims=1)
plot(xmesh,vparrayvec,xaxis="x (m)",yaxis="A_y(x)",title="vector potential component")

In [ ]:
bfluxarray[1](xmesh[1],m[1],xmesh[1])

In [ ]:
# evaluate magnetic flux as sum of contribution over N elements
bfluxarrayvec = sum([bfluxarray[k].(xmesh,Ref(m),Ref(xmesh)) for k=1:N],dims=1)
plot(xmesh,bfluxarrayvec,xaxis="x (m)",yaxis="B_z(x)",title="magnetic flux component")

## Section 6: Implicit Construction: Magnetic Source Unknown - Vector Potential and Magnetic Flux Defined in Terms of Expansion of Numerical Approximation to Magnetization
Creating a function mapping m to a vector-valued output is the <b>key</b> here. This function is the magnetic flux colllocated in the nodes of the mesh.  

In [ ]:
blabla = m->sum([bfluxarray[k](xmesh,m,xmesh) for k=1:N],dims=1)



In [ ]:
mm = ones(size(xmesh));

In [ ]:
blabla(mm)

In [ ]:
methods(bfluxarray[1])

In [ ]:
bfluxarray[1](0.1,0.1,0.1)

In [ ]:
function bfluxarrayvec(m)
    results = zeros(size(m))
    N = length(m)-1 
    for k=1:N 
       results +=  
    end 
    return result
    
sum([bfluxarray[k].(xmesh,m,xmesh) for k=1:N],dims=1)

In [ ]:
bfluxarrayvec2(mm)

In [ ]:
[bfluxarray[1](xi,mm,xi) for xi in xmesh]

In [ ]:
bfluxsamp(m) = sum([bfluxarray[k](xi,m,xi) for xi in xmesh for k=1:N])

In [ ]:
# compute the Jacobian  
mm = ones(size(xmesh))
bfluxsamp(mm)
##ForwardDiff.jacobian(bfluxsamp, mm) 

## Section 7: Memory Allocation for Automatic Differentiation of Jacobian 

In [ ]:
M = @SVector([10.0, 0.0, 0.0])
r = Vector([2.0, 1.0, 0.0]) 
y = Vector([0., 0., 0.])

function f!(y, r, M)
    y[1] = M * r[3]/norm(r)
    y[2] = M * r[1]/norm(r)
    y[3] = M * r[2]/norm(r)
end

function curl_of_function(f!, r)
    out=zeros(3)
    
    config = ForwardDiff.JacobianConfig(f!, out, r, ForwardDiff.Chunk{3}())
    Jac = Matrix{Float64}(undef, length(out), length(r))

    ForwardDiff.jacobian!(Jac, f!, out, r, config)
    curl_x = Jac[3,2] - Jac[2,3]
    curl_y = Jac[1,3] - Jac[3,1]
    curl_z = Jac[2,1] - Jac[1,2]
    return @SVector[curl_x, curl_y, curl_z]
end

@btime curl_of_function(f!, [2.0, 1.0, 0.0])

In [ ]:
y

In [ ]:
f!(y,r,M)

In [ ]:
function f(x,y,z, r)
    r_prime = [x,y,z]
    #r_prime = [x,y,z]
    return (r - r_prime)/norm(r - r_prime)
end

In [ ]:
g(x,y,r) = quadgk(z -> f(x, y, z, r), 0, 1)[1]
h(x, r) = quadgk(y -> g(x, y, r), 0, 1)[1]
l(r) =  quadgk(x -> h(x, r), 0, 1)[1]

In [ ]:
function curl_of_integral(f, r)

    out=zeros(3)
    
    config = ForwardDiff.JacobianConfig(l_wrapper!, out, r, ForwardDiff.Chunk{1}())
    Jac = Matrix{Float64}(undef, length(out), length(r))
    
    @btime ForwardDiff.jacobian!(l_wrapper!, r, $config)
    
    curl_x = Jac[3,2] - Jac[2,3]
    curl_y = Jac[1,3] - Jac[3,1]
    curl_z = Jac[2,1] - Jac[1,2]
    
    return [curl_x, curl_y, curl_z]
end     

In [ ]:
curl_of_integral(f, [2.0, 1.0, 0.0])